In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binance.client import Client
import datetime
from tqdm.auto import tqdm

In [2]:
api_key = ('5C9nlellYmjpT7hGY1RwtCqafHEiB718aYptf2gH233vVxAIc1XG1WEgwuss4sKb')
api_secret = ('qEOCX9cOU4n3WMex3PTi8U8IlTEp2DQfKKlZYvHTFaQ3GB0AUTeos3P2Iff5CipA')
client = Client(api_key, api_secret)

In [17]:
ASSETS = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT', 'USDCUSDT', 'DOGEUSDT', 'ADAUSDT', 'TRXUSDT']
#https://coinmarketcap.com/ top 10 currencies

In [20]:
def GetHistoricalData(start, finish):
    df = pd.DataFrame()
    for asset in tqdm(ASSETS):
        candle = client.get_historical_klines(asset, Client.KLINE_INTERVAL_15MINUTE, str(start), str(finish))
        df_new = pd.DataFrame(candle, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
        df['dateTime'] = df_new.dateTime
        df[asset] = df_new.open
        df.dateTime = pd.to_datetime(df.dateTime, unit='ms')

    print(df.shape)
    return df

In [21]:
%%time
untilThisDate = datetime.datetime(2025, 4, 1)
sinceThisDate = datetime.datetime(2020, 1, 1)
df = GetHistoricalData(sinceThisDate, untilThisDate)

  0%|          | 0/8 [00:00<?, ?it/s]

(183880, 9)
CPU times: total: 6.48 s
Wall time: 16min 15s


In [23]:
df.set_index('dateTime', inplace = True)

In [190]:
for col in df.columns:
    df[col] = df[col].astype('float32')

In [1]:
df.head()

NameError: name 'df' is not defined

In [212]:
df.to_csv('..//data//ts_data202504-202001.csv')

In [80]:
text_data_paths = ['..//data//crypto_news_2025-2020-06.csv',
                  '..//data//decrypt_with_years.csv',
                  '..//data//cryptobriefing_2025_2019_04_03.csv']

In [84]:
df_list = []
for path in text_data_paths:
    df_news = pd.read_csv(path)
    df_news = df_news[['date', 'header']]
    print(df_news.date.iloc[-1])
    df_news['date'] = pd.to_datetime(df_news.date, format = 'mixed')
    df_news.set_index('date', inplace = True)
    df_news = df_news.tz_localize(None)
    df_list.append(df_news)

2020-06-02T11:00:13+00:00
Feb 23, 8:32 pm, 2023
2019-04-03T18:59:53+00:00


In [85]:
df_text = pd.concat(df_list).sort_index()
df_text.head()

,header
date,
2019-04-03 18:59:53,Open Analysis: Will Bitcoin Jump Again?
2019-04-03 19:34:21,Open Smartlands Steps Closer To A Regulated Se...
2019-04-03 21:07:47,Open Nash Exchange: The Compliant Crypto DEX O...
2019-04-03 22:08:03,Open Put Your Crypto To Work: Five Ways To Ear...
2019-04-03 23:42:47,Open What Is Loom Network? Introduction to LO...


In [113]:
print(df_text.loc[df_text.index > '2025-01-01'].shape)
print(df.loc[df.index > '2025-01-01'].shape)

(4273, 1)
(8640, 8)


In [153]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModel
from transformers import get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup
import torch.optim as optim

In [196]:
class CombDataset(Dataset):
    def __init__(self, ts_data, text_data, window_size = 50, news_amount = 20):
        self.ts_data = ts_data
        self.text_data = text_data
        self.window_size = window_size
        self.news_amount = news_amount
        self.tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

    def __len__(self):
        return len(self.ts_data) - self.window_size

    def __getitem__(self, idx):
        latest_date = self.ts_data.index[idx + self.window_size - 1]
        #print(latest_date)
        latest_news = ' [SEP] '.join(self.text_data.loc[self.text_data.index <= latest_date][-self.news_amount:].header.values)
        ts = self.ts_data.iloc[idx:idx + self.window_size].values
        target = self.ts_data.iloc[idx + self.window_size].values

        return torch.Tensor(ts), self.tokenizer(latest_news, max_length = 512, padding = 'max_length', truncation = True) ,torch.Tensor(target)

In [197]:
train_dataset = CombDataset(df.loc[df.index < '2024-06-01'], 
                                df_text.loc[df_text.index < '2024-06-01'])
val_dataset = CombDataset(df.loc[df.index >= '2024-06-01'], 
                                df_text.loc[df_text.index >= '2024-06-01'])

In [173]:
df.loc[df.index < '2024-06-01'].index[25]

Timestamp('2020-01-01 06:15:00')

In [158]:
len(train_dataset)

154645

In [159]:
len(val_dataset)

29135

In [143]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.2, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [208]:
class DenchickNet(nn.Module):
    def __init__(self, input_dim=8, d_model=256, nhead=8, num_layers=2, dropout=0.2):
        super(DenchickNet, self).__init__()     

        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.text_encoder = AutoModel.from_pretrained('google-bert/bert-base-uncased')
        self.fc1 = nn.Linear(d_model + self.text_encoder.config.hidden_size, d_model)
        self.fc2 = nn.Linear(d_model, input_dim)
        
        self.relu = F.relu
        self.dropout = nn.Dropout(dropout)

    def forward(self, ts, input_token_ids, token_attention_mask):
        ts = self.encoder(ts)
        ts = self.pos_encoder(ts)
        ts = self.transformer_encoder(ts)

        outputs = self.text_encoder(input_ids=input_token_ids, attention_mask=token_attention_mask)
        pooled_output = outputs.pooler_output

        x = torch.cat([ts[:, -1, :], pooled_output], dim = 1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [150]:
def MAPE(output, target):
  return torch.mean(torch.abs((target - output) / target))

In [210]:
device = 'cuda'
model = DenchickNet().to(device)

In [211]:
loader = DataLoader(train_dataset, shuffle=True, batch_size = 128)
eval_loader = DataLoader(val_dataset, shuffle = False, batch_size = 128)

optimizer = optim.Adam(model.parameters(), lr = 1e-4, eps = 1e-8)
n_epochs = 50
total_steps = len(loader) * n_epochs

scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
MSE = nn.MSELoss()
MAE = nn.SmoothL1Loss()


streak = 0
for epoch in range(n_epochs):
    model.train()
    for ts_batch, text_batch, y_batch in tqdm(loader, desc=f'Training epoch {epoch}:'):
        ts_batch = ts_batch.to(device)
        input_token_ids = torch.stack(text_batch['input_ids'], dim = 0).to(device)
        token_attention_mask = torch.stack(text_batch['attention_mask'], dim = 0).to(device)
        y_batch = y_batch.to(device)
        
        y_pred = model(ts_batch, input_token_ids, token_attention_mask)
        loss = MAE(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    model.eval()
    test_rmse = []
    test_mae = []
    test_mape = []
    with torch.no_grad():
        for ts_batch, text_batch, y_batch in eval_loader:
            ts_batch = ts_batch.to(device)
            input_token_ids = text_batch['input_ids'].to(device)
            token_attention_mask = text_batch['attention_mask'].to(device)
            y_batch = y_batch.to(device)

            y_pred = model(X_batch)
            test_rmse.append(torch.sqrt(MSE(y_pred, y_batch)).item())
            test_mae.append(MAE(y_pred, y_batch).item())
            test_mape.append(MAPE(y_pred, y_batch).item())

    av_mae = sum(test_mae)/len(test_mae)
    if round(av_mae, 3) >= round(prev_mae, 3):
        streak += 1
        if streak >= 3:
            av_rmse = sum(test_rmse)/len(test_rmse)
            av_mape = sum(test_mape)/len(test_mape)
            print("Epoch %d: test RMSE %.6f test MAE %.6f test MAPE %.6f streak %d" % (epoch, av_rmse, av_mae, av_mape, streak))
            break
    else:
        streak = 0
    prev_mae = av_mae
    av_rmse = sum(test_rmse)/len(test_rmse)
    av_mape = sum(test_mape)/len(test_mape)
    print("Epoch %d:test RMSE %.6f test MAE %.6f test MAPE %.6f streak %d" % (epoch, av_rmse, av_mae, av_mape, streak))

Training epoch 0::   0%|          | 0/1209 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 6.00 GiB total capacity; 11.65 GiB already allocated; 0 bytes free; 11.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF